# Load All-Star Data From NBA API

In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
def boxscoretraditionalv2(GameID,EndPeriod='10',EndRange='28800',RangeType='0',StartPeriod='1',StartRange='0'):
    url = 'http://stats.nba.com/stats/boxscoretraditionalv2?'
    api_param = {
        'EndPeriod' : EndPeriod,
        'EndRange' : EndRange,
        'GameID' : GameID,
        'RangeType' : RangeType,
        'StartPeriod' : StartPeriod,
        'StartRange' : StartRange,
        }
    u_a = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.82 Safari/537.36"
    response = requests.get(url,params=api_param,headers={"USER-AGENT":u_a})
    data = response.json()
    return pd.DataFrame(data['resultSets'][0]['rowSet'],columns=data['resultSets'][0]['headers'])

def commonallplayers(currentseason=0,leagueid='00',season='2015-16'):
    url = 'http://stats.nba.com/stats/commonallplayers?'
    api_param = {
        'IsOnlyCurrentSeason' : currentseason,
        'LeagueID' : leagueid,
        'Season' : season,             
    }
    u_a = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.82 Safari/537.36"
    response = requests.get(url,params=api_param,headers={"USER-AGENT":u_a})
    data = response.json()
    return pd.DataFrame(data['resultSets'][0]['rowSet'],columns=data['resultSets'][0]['headers'])

def seasons_string(start,end):
    '''
    creates a list of NBA seasons from start-end
    '''
    years = np.arange(start,end+1)
    seasons = []
    for year in years:
        string1 = str(year)
        string2 = str(year+1)
        season = '{}-{}'.format(string1,string2[-2:])
        seasons.append(season)
    return seasons

def gamelog(counter = 1000,datefrom='',dateto='',direction='DESC',leagueid='00',
            playerorteam='T',season='2015-16',seasontype='Regular Season',sorter='PTS'):   
    url = 'http://stats.nba.com/stats/leaguegamelog?'
    api_param = {
        'Counter' : counter,
        'DateFrom' :  datefrom,
        'DateTo' : dateto,
        'Direction' : direction,
        'LeagueID' : leagueid,
        'PlayerOrTeam' : playerorteam,
        'Season' : season,
        'SeasonType' : seasontype,
        'Sorter' : sorter,              
    }
    u_a = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.82 Safari/537.36"
    response = requests.get(url,params=api_param,headers={"USER-AGENT":u_a})
    data = response.json()
    return pd.DataFrame(data['resultSets'][0]['rowSet'],columns=data['resultSets'][0]['headers'])

## Get game logs for all all-star games since 1978

In [3]:
games = []
for season in seasons_string(1978,2017):
    df  = gamelog(season=season,seasontype='All Star')
    games.append(df)
games = pd.concat(games,ignore_index=True)
games.head(10)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,31978,1610616834,WST,West NBA All Stars West,0037800029,1979-02-04,WST @ EST,W,240,56,...,30,45,25,10,3,13,24,134,5,0
1,31978,1610616833,EST,East NBA All Stars East,0037800029,1979-02-04,EST vs. WST,L,240,50,...,31,53,22,8,3,16,26,129,-5,0
2,31979,1610616833,EST,East NBA All Stars East,0037900001,1980-02-03,EST vs. WST,W,265,58,...,31,62,34,20,9,30,30,144,8,0
3,31979,1610616834,WST,West NBA All Stars West,0037900001,1980-02-03,WST @ EST,L,265,53,...,31,55,34,17,16,29,32,136,-8,0
4,31980,1610616833,EST,East NBA All Stars East,0038000001,1981-02-01,EST vs. WST,W,240,49,...,28,48,30,15,5,17,23,123,3,0
5,31980,1610616834,WST,West NBA All Stars West,0038000001,1981-02-01,WST @ EST,L,240,46,...,34,51,22,10,7,22,28,120,-3,0
6,31981,1610616833,EST,East NBA All Stars East,0038100002,1982-01-31,EST vs. WST,W,240,51,...,44,57,30,10,9,21,28,120,2,0
7,31981,1610616834,WST,West NBA All Stars West,0038100002,1982-01-31,WST @ EST,L,240,50,...,37,60,25,13,10,16,27,118,-2,0
8,31982,1610616833,EST,East NBA All Stars East,0038200001,1983-02-13,EST @ WST,W,240,57,...,31,52,36,17,6,22,15,132,9,0
9,31982,1610616834,WST,West NBA All Stars West,0038200001,1983-02-13,WST vs. EST,L,240,55,...,25,37,37,17,7,24,18,123,-9,0


## Load box score for all the all-star games

In [5]:
df = []
ugames = np.unique(games['GAME_ID'].values)
for gameid in ugames:
    temp = boxscoretraditionalv2(gameid)
    temp['SEASON'] = season
    df.append(temp)
df = pd.concat(df,ignore_index=True)
df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,SEASON
0,0030000001,1610616834,WST,West NBA All Stars,1495,Tim Duncan,F,,28:14,5.0,...,10.0,14.0,1.0,2.0,1.0,2.0,1.0,14.0,8,2017-18
1,0030000001,1610616834,WST,West NBA All Stars,185,Chris Webber,F,,28:57,6.0,...,5.0,9.0,3.0,1.0,0.0,1.0,2.0,14.0,9,2017-18
2,0030000001,1610616834,WST,West NBA All Stars,708,Kevin Garnett,C,,27:20,7.0,...,3.0,4.0,4.0,1.0,3.0,2.0,0.0,14.0,10,2017-18
3,0030000001,1610616834,WST,West NBA All Stars,977,Kobe Bryant,G,,29:31,9.0,...,2.0,4.0,7.0,1.0,0.0,3.0,3.0,19.0,0,2017-18
4,0030000001,1610616834,WST,West NBA All Stars,467,Jason Kidd,G,,29:40,4.0,...,4.0,4.0,2.0,5.0,0.0,5.0,3.0,11.0,-3,2017-18


In [12]:
df['PLAYER'] = zip(df['PLAYER_NAME'],df['PLAYER_ID']) # create a unique player column from name + id

# Groupby to count how many times a player appeared in the games
g = df.groupby('PLAYER').size()
g = pd.DataFrame(g,index=g.index,columns=['AllStar_count'])
g['PLAYER_NAME'],g['PLAYER_ID'] = zip(*g.index)
g = g.sort_values(by='AllStar_count',ascending=False).reset_index(drop=True)
g.head(10)

,AllStar_count,PLAYER_NAME,PLAYER_ID
0,17,Kobe Bryant,977
1,15,Tim Duncan,1495
2,14,Kevin Garnett,708
3,14,LeBron James,2544
4,13,Michael Jordan,893
5,13,Dirk Nowitzki,1717
6,13,Shaquille O'Neal,406
7,12,Karl Malone,252
8,12,Hakeem Olajuwon,165
9,11,Kareem Abdul-Jabbar,76003
